<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/draft7_Untitled23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# 📦 Imports
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta
import numpy as np

# 🔑 Replace with your own OpenWeatherMap API Key if needed
API_KEY = '7cf20335110caaf78db0fecb31852d45'

# 🌐 Fetch weather data
def get_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

# 🌡️ Improved Temperature Chart with confidence band & better formatting
def create_temperature_visualisation(data, city):
    temps = [entry['main']['temp'] for entry in data['list']]
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    temps = np.array(temps)

    # Confidence interval example ±1°C
    lower_bound = temps - 1
    upper_bound = temps + 1

    plt.figure(figsize=(14,6))
    plt.plot(times, temps, marker='o', linestyle='-', color='#FF6F61', linewidth=2, label='Temperature (°C)')
    plt.fill_between(times, lower_bound, upper_bound, color='#FF6F61', alpha=0.2, label='Confidence band (±1°C)')

    # Annotate peaks for clarity (local maxima and minima)
    for i in range(1, len(temps)-1):
        if temps[i] > temps[i-1] and temps[i] > temps[i+1]:
            plt.annotate(f"{temps[i]:.1f}°C", (times[i], temps[i]), textcoords="offset points", xytext=(0,10), ha='center', fontsize=9, color='#9b2d24')
        elif temps[i] < temps[i-1] and temps[i] < temps[i+1]:
            plt.annotate(f"{temps[i]:.1f}°C", (times[i], temps[i]), textcoords="offset points", xytext=(0,-15), ha='center', fontsize=9, color='#9b2d24')

    plt.title(f"5-Day Temperature Forecast for {city.title()}", fontsize=18)
    plt.xlabel("Date & Time", fontsize=14)
    plt.ylabel("Temperature (°C)", fontsize=14)

    # Better date formatting on x-axis
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=12))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%a %H:%M'))
    plt.gca().xaxis.set_minor_locator(mdates.HourLocator(interval=3))
    plt.xticks(rotation=45, fontsize=10)
    plt.yticks(fontsize=12)

    plt.grid(True, linestyle='--', alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()

# 🌧️ Improved Precipitation Chart with color gradient and better layout
def create_precipitation_visualisation(data, city):
    rain_chances = []
    times = []

    for entry in data['list']:
        rain = entry.get('rain', {}).get('3h', 0)
        rain_chances.append(rain)
        times.append(datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S"))

    rain_chances = np.array(rain_chances)
    plt.figure(figsize=(14,6))

    # Use a colormap from light blue to dark blue based on rain intensity
    cmap = plt.get_cmap('Blues')
    norm = plt.Normalize(rain_chances.min(), rain_chances.max())

    bars = plt.bar(times, rain_chances, width=0.12, color=cmap(norm(rain_chances)), edgecolor='black')

    # Annotate bars with significant rain amounts only (above 0.5 mm)
    for bar, value in zip(bars, rain_chances):
        if value > 0.5:
            plt.text(bar.get_x() + bar.get_width()/2, value + 0.1, f"{value:.1f} mm",
                     ha='center', fontsize=9, color='navy')

    plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)", fontsize=18)
    plt.xlabel("Date & Time", fontsize=14)
    plt.ylabel("Rainfall (mm per 3h)", fontsize=14)

    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=12))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%a %H:%M'))
    plt.gca().xaxis.set_minor_locator(mdates.HourLocator(interval=3))
    plt.xticks(rotation=45, fontsize=10)
    plt.yticks(fontsize=12)

    plt.grid(axis='y', linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()

# 🔍 Parse user weather question
def parse_weather_question(question):
    match = re.search(r"(rain|snow|clear|cloudy).* in (\w+(?: \w+)*) (today|tomorrow)?", question.lower())
    if match:
        condition = match.group(1)
        city = match.group(2)
        day = match.group(3) or 'today'
        return {'condition': condition, 'city': city, 'day': day}
    return None

# 🤖 Generate response to question
def generate_weather_response(parsed):
    if not parsed:
        return "⚠️ Sorry, I couldn't understand your question. Try asking like: 'Will it rain in Paris tomorrow?'"

    condition = parsed['condition']
    city = parsed['city']
    day = parsed['day']

    try:
        data = get_weather_data(city)
        target_date = datetime.utcnow().date()
        if day == 'tomorrow':
            target_date += timedelta(days=1)

        found = False
        for entry in data['list']:
            entry_date = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date()
            weather_desc = entry['weather'][0]['description'].lower()
            if entry_date == target_date and condition in weather_desc:
                found = True
                break

        if found:
            return f"✅ Yes, it looks like it will be {condition} in {city.title()} {day}."
        else:
            return f"❌ No, it doesn't look like it will be {condition} in {city.title()} {day}."

    except Exception as e:
        return f"❌ Could not retrieve weather for {city.title()}: {e}"

# 🧑‍💻 Interactive Widget UI
def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:')
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:')
    output_area = widgets.Output()

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            try:
                data = get_weather_data(city_input.value)
                create_temperature_visualisation(data, city_input.value)
                create_precipitation_visualisation(data, city_input.value)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            parsed = parse_weather_question(question_input.value)
            response = generate_weather_response(parsed)
            print(response)

    forecast_btn = widgets.Button(description="📊 Show Forecast Charts", button_style='info')
    question_btn = widgets.Button(description="🧠 Answer My Question", button_style='success')

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)

    display(widgets.VBox([
        widgets.HTML(value="<h2>🌤️ Welcome to your Interactive Weather Assistant!</h2>"),
        question_input, question_btn,
        widgets.HTML(value="<hr>"),
        city_input, forecast_btn,
        output_area
    ]))

# 🚀 Launch the Assistant
interactive_ui()
